### SOCIETE GENERALE
The aim of this challenge is to predict the Crude Oil production's trend base on the previous year Crude Oil data.

That would help the compagny to predict the revenue generated by the Trade Finance business line

Understanding the variation of the product per region helps in predicting the Oil price trend in these regions.

The objective is to predict the probability of increase of Crude Oil production per quarter per contry.

TODO : 
    1. Data analysis
    2. Neural Network
    3. RandomForestClassifier with bagging and random feature subset
    4. 4
    5. 5

In [1]:
import json
import matplotlib
import warnings
import pandas as pd
import numpy as np
import math
from useful import useful
from matplotlib import pyplot as plt
from IPython.core.pylabtools import figsize
from sklearn.metrics import roc_auc_score

warnings.simplefilter('ignore')
root = r"/Users/Kenneth-Aristide/anaconda3/bin/python_prog/ML/styles/bmh_matplotlibrc.json"
s = json.load(open(root))
matplotlib.rcParams.update(s)
%matplotlib inline

In [2]:
def normalized_data(data, columns):
    """
    Convenience function: 
        Normalized data for a Neural Network
    """
    for col in columns:
        data[col] = (data[col] - data[col].mean()) / data[col].std()
        
    return data


def data_cleaner(root):
    """
    Convenience function : 
        load the data
        find if they're nan value in columns, if True fill with the median
        create boolean indicator for each month and county
        return a test and the new dataset
    """
    data = pd.read_csv(root, sep = ';')
    bool_object = data.isnull().any()
    bool_object = bool_object.loc[bool_object == True]
    nan_list = bool_object.index
    for col in nan_list:
        data[col] = data[col].fillna(data[col].median())
        
    dummies_months = pd.get_dummies(data['month'], prefix = 'month')
    dummies_counties = pd.get_dummies(data['country'], prefix = 'county')
    _data = pd.concat([data, dummies_months, dummies_counties], axis = 1)
    _data = _data.drop(['country', 'month'], axis = 1)
    
    return _data.isnull().any().any(), _data, data


root = '/Users/Kenneth-Aristide/anaconda3/bin/python_prog/ML_Training/data/BankSG.csv'
test, bank_data, bank = data_cleaner(root)

# load the label
bank_label = pd.read_csv('/Users/Kenneth-Aristide/anaconda3/bin/python_prog/ML_Training/data/SGLabels.csv',
                         sep = ';')

# add the label to the data
bank_data['actual_labels'] = bank_label.Target
print(test)
bank_data.tail()

False


,ID,1_diffClosing stocks(kmt),1_diffExports(kmt),1_diffImports(kmt),1_diffRefinery intake(kmt),1_diffWTI,1_diffSumClosing stocks(kmt),1_diffSumExports(kmt),1_diffSumImports(kmt),1_diffSumProduction(kmt),...,county_68,county_69,county_70,county_71,county_72,county_73,county_74,county_75,county_76,actual_labels
10154,ID10155,0.0000,0.0000,0.0000,0.0000,6.80,-469.1997,-6034.6239,-1237.4344,1475.3527,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
10155,ID10156,209.0000,0.0000,525.0000,-506.0000,-3.41,5567.0593,2212.8782,4247.3021,14104.9032,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
10156,ID10157,23.0000,0.0000,20.0000,12.0000,5.82,4256.6040,-6163.9167,5479.7160,-174.7942,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
10157,ID10158,-361.0000,0.0000,-746.0000,-13.0000,4.89,644.1414,-4513.0417,-21586.6926,-1420.9013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
10158,ID10159,-252.4286,-487.1428,8.8571,1049.5714,-0.37,7872.1638,1455.8212,8591.6520,4178.0130,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [3]:
col_to_normalized = list(bank.columns)
col_to_normalized.remove('ID')
col_to_normalized.remove('month')
col_to_normalized.remove('country')
bank_data = normalized_data(bank_data, col_to_normalized)

In [37]:
_root = '/Users/Kenneth-Aristide/anaconda3/bin/python_prog/ML_Training/data/Test.csv'
_data = pd.read_csv(_root, sep = ';')
test, bank_test, bankTest = data_cleaner(_root)
bank_test = normalized_data(bank_test, col_to_normalized)

In [38]:
bank_test = bank_test[best_features_list]
bank_test['ones'] = np.ones(bank_test.shape[0])
X_new = bank_test.values
X_new.shape

(2000, 209)

#### Let's find the top  best predictors for the crude Oil production

In [5]:
features = list(bank_data.columns)
features.remove('ID')
features.remove('actual_labels')

best_features_dict, best_features_list = useful.find_best_column(bank_data, 'actual_labels', features)
print(best_features_list[-10:])

['1_diffExports(kmt)', '7_diffSumProduction(kmt)', '12_diffSumExports(kmt)', '7_diffSumRefinery intake(kmt)', '1_diffSumImports(kmt)', '12_diffSumRefinery intake(kmt)', '1_diffSumRefinery intake(kmt)', '12_diffSumProduction(kmt)', '1_diffSumExports(kmt)', '1_diffSumProduction(kmt)']


### Autocorelation check
Here I would study the autocorelation between all the variable describing one month of data

In [6]:
new_features = [elem for elem in best_features_list if 'diffSum' in elem or 'month' in elem or 'county' in elem]
new_features_bis = new_features.copy()
new_features_bis.append('actual_labels')

In [17]:
#new_features.remove('actual_labels')
_bank_data = bank_data[best_features_list[-199:]]
_bank_data['ones'] = np.ones(bank_data.shape[0])

X = _bank_data.values
y = bank_data['actual_labels'].values.astype(int)
print(X.shape, y.shape)

(10159, 200) (10159,)


In [41]:
# randomly initialize the thetas (weights)
theta_init = np.random.normal(0, 0.01, size = (X.shape[1], 1))

def sigmoid_activation(x, theta):
    """
    Convenience function:
        compute the activation of the NNet
    """
    x = np.asarray(x)
    theta = np.asarray(theta)
    return 1 / (1 + np.exp(-np.dot(theta.T, x)))

In [39]:
class NNet3:
    def __init__(self, learning_rate=0.5, maxepochs=1e4, convergence_thres=1e-5):
        """
        Convenience function:
            initialize the NNet parameters
        """
        self.learning_rate = learning_rate
        self.maxepochs = int(maxepochs)
        self.convergence_thres = 1e-5
        
    def multiplecost(self, X, y):
        l1, l2 = self.feedforward(X)
        inner = y * np.log(l2) + (1 - y) * np.log(1- l2)
        return -np.mean(inner)
    
    def sigmoid_activation(self, X, theta):
        X = np.asarray(X)
        theta = np.asarray(theta)
        return 1 / (1 + np.exp(-np.dot(theta.T, X)))
    
    def feedforward(self, X):
        l1 = self.sigmoid_activation(X.T, self.theta0).T
        l1 = np.column_stack([np.ones(l1.shape[0]), l1])
        l2 = self.sigmoid_activation(l1.T, self.theta1)
        return l1, l2
    
    def predict(self, X):
        _, y = self.feedforward(X)
        return y
    
    def learn(self, X, y):
        nobs, ncols = X.shape
        self.theta0 = np.random.normal(0, 0.01, size = (ncols, 500))
        self.theta1 = np.random.normal(0, 0.01, size = (501, 1))

        self.costs = []
        cost = self.multiplecost(X, y)
        self.costs.append(cost)
        costprev = cost + self.convergence_thres + 1
        
        counter = 0
        
        for counter in np.arange(self.maxepochs):
            l1, l2 = self.feedforward(X)
            
            # Backpropagation
            l2_delta = (y - l2) * l2 * (1 - l2)
            l1_delta =  l2_delta.T.dot(self.theta1.T) * l1 * (1 - l1)
            
            # Update parameters
            self.theta1 += l1.T.dot(l2_delta.T) / nobs * self.learning_rate
            self.theta0 += X.T.dot(l1_delta)[:, 1:] / nobs * self.learning_rate
            
            counter += 1
            costprev = cost
            cost = self.multiplecost(X, y)
            self.costs.append(cost)
            if np.abs(costprev - cost) < self.convergence_thres and counter > 500:
                break
        

In [57]:
theta_1_init = np.random.normal(0, 0.01, size = (X.shape[1], 500))
a2 = sigmoid_activation(X.T, theta_1_init)
theta_2_init = np.random.normal(0, 0.01, size = (X.shape[0], 500))
a3 = sigmoid_activation(a2.T, theta_2_init)
theta_3_init = np.random.normal(0, 0.01, size = (500, 1))
a4 = sigmoid_activation(a3.T, theta_3_init)

In [48]:
X.shape

(10159, 200)

In [50]:
theta_1_init.shape

(200, 500)

In [52]:
a2.shape

(500, 10159)

In [55]:
a3.shape

(500, 500)

In [58]:
a4.shape

(1, 500)

In [40]:
import math
split = math.floor(X.shape[0] * .8)
X_train = X[:split]
y_train = y[:split]
X_test = X[split:]
y_test = y[split:]


learning_rate = 0.5
maxepochs = 10000
convergence_thres = 0.00001

_model = NNet3(learning_rate = learning_rate, maxepochs = maxepochs, convergence_thres = convergence_thres)

%time _model.learn(X_train, y_train)
probabilities_test = _model.predict(X_test)
probabilities_train = _model.predict(X_train)
auc_train = roc_auc_score(y_train, probabilities_train[0])
auc_test = roc_auc_score(y_test, probabilities_test[0])
print("Train score : {0} \n Test score : {1}".format(auc_train, auc_test))

CPU times: user 1h 29min 52s, sys: 19min 19s, total: 1h 49min 12s
Wall time: 1h 1min 6s
Train score : 0.8293712596094763 
 Test score : 0.8159876120930579


In [23]:
probabilities = _model.predict(X_new)

In [ ]:
def singlecost(X, y, theta):
    """
    Convenience function :
        compute the cost for a single observation
    """
    h = sigmoid_activation(X.T, theta)
    return - np.mean(y * np.log(h) + (1 - y) * np.log(1 - h))

cost_0 = singlecost(X[0], y[0], theta_init)
cost_0

In [ ]:
# let's compute the gradients for each parameter of the model
grads = np.zeros(theta_init.shape)
n = X.shape[0]

for j , obs in enumerate(X):
    h = sigmoid_activation(obs, theta_init)
    delta = (y[j] - h) * h * (1 - h ) * obs
    grads += delta[:, np.newaxis] / n

In [ ]:
# two layer neural network
learning_rate = 0.1
maxepochs = 10000
convergence_thres = 0.0001

def learn(X, y, theta, learning_rate, maxepochs, convergence_thres):
    """
    Convenience function:
        learn a two layer Neural Networks
    """
    costs = []
    cost = singlecost(X, y, theta)
    costprev = cost + convergence_thres + 0.01
    counter = 0
    
    for counter in np.arange(maxepochs):
        grads = np.zeros(theta.shape)
        for j, obs in enumerate(X):
            h = sigmoid_activation(obs, theta)
            delta = (y[j] - h)* h *(1 - h) * obs
            grads += delta[:, np.newaxis] / X.shape[0]
            
        theta += grads * learning_rate
        counter += 1
        costprev = cost
        cost = singlecost(X, y, theta)
        costs.append(cost)
        if np.abs(costprev - cost) < convergence_thres:
            break
            
            
    figsize(12.5, 5)
    plt.plot(costs)
    plt.title("Convergence of the cost function for a two layers NNets")
    plt.ylabel("J($\Theta$)")
    plt.xlabel("Iteration")
    plt.legend();
    return theta, costs

theta, costs = learn(X, y, theta_init, learning_rate, maxepochs, convergence_thres)

In [ ]:
def feedforward_4(x, theta_0, theta_1, theta_2):
    """
    Convenience function
        compute the output of a 4 layer NNets
    """
    a1 = sigmoid_activation(x.T, theta_0).T
    a1 = np.column_stack([np.ones(a1.shape[0]), a1])
    a2 =  sigmoid_activation(a1.T, theta_1).T
    a2 = np.column_stack([np.ones(a2.shape[0]), a2])
    out = sigmoid_activation(a2.T, theta_2)
    return a1, a2, out

theta0_init = np.random.normal(0, 0.01, size = (X.shape[1], (X.shape[1] - 1)))
theta1_init = np.random.normal(0, 0.01, size = (X.shape[1], (X.shape[1] - 1)))
theta2_init = np.random.normal(0, 0.01, size = (X.shape[1], 1))

l1, l2, l3 = feedforward_4(X, theta0_init, theta1_init, theta2_init)

In [ ]:
l3_delta = l3 - y
l2_delta = l3_delta.T.dot(theta2_init.T) * l2 * (1 - l2)
l1_delta = (l2_delta.dot(theta1_init)).T.dot((l1 * (1 - l1)))

In [ ]:
((l1 * l2_delta) / X.shape[0]).shape

In [ ]:
theta1_init .shape

In [ ]:
def multiplecost(X, y, theta0, theta1, theta2, lambda_):
    """
    Convenience function:
        compute the cost of a 4 layer NNets
    """
    _, __, h = feedforward_4(X, theta0, theta1, theta2)
    reg_term = (sum(sum(theta0_init ** 2)) + sum(sum(theta1_init ** 2)) + \
                sum(sum(theta2_init ** 2)) ) * (lambda_ / (2 * X.shape[0]))
    return -np.mean(y * np.log(h) + (1 - y) * np.log(1 - h)) + reg_term

c = multiplecost(X, y, theta0_init, theta1_init, theta2_init, 1)
c

In [ ]:
def feedforward_3(x, theta_0, theta_1):
    """
    Convenience function
        compute the output of a 3 layer NNets
    """
    a1 = sigmoid_activation(x.T, theta_0).T
    a1 = np.column_stack([np.ones(a1.shape[0]), a1])
    out = sigmoid_activation(a1.T, theta_1)
    return out

theta0_init = np.random.normal(0, 0.01, size = (X.shape[1], (X.shape[1] - 1)))
theta1_init = np.random.normal(0, 0.01, size = (X.shape[1], 1))


h = feedforward_3(X, theta0_init, theta1_init)
h                             

In [ ]:
def _multiplecost(X, y, theta0, theta1):
    """
    Convenience function:
        compute the cost of a 3 layer NNets
    """
    h = feedforward_3(X, theta0, theta1)
    return -np.mean(y * np.log(h) + (1 - y) * np.log(1 - h))

c = _multiplecost(X, y, theta0_init, theta1_init)
c

In [ ]:
class NNet4:
    def __init__(self, learning_rate=0.5, maxepochs=1e4, convergence_thres=1e-5):
        """
        Convenience function:
            initialize the NNet parameters
        """
        self.learning_rate = learning_rate
        self.maxepochs = int(maxepochs)
        self.convergence_thres = 1e-5
        
    def multiplecost(self, X, y):
        l1, l2, l3 = self.feedforward(X)
        inner = y * np.log(l3) + (1 - y) * np.log(1- l3)
        return -np.mean(inner)
    
    def feedforward(self, X):
        l1 = sigmoid_activation(X.T, self.theta0).T
        l1 = np.column_stack([np.ones(l1.shape[0]), l1])
        l2 = sigmoid_activation(l1.T, self.theta1).T
        l2 = np.column_stack([np.ones(l2.shape[0]), l2])
        l3 = sigmoid_activation(l2.T, self.theta2)
        return l1, l2, l3
    
    def predict(self, X):
        _,__, y = self.feedforward(X)
        return y
    
    def learn(self, X, y):
        nobs, ncols = X.shape
        self.theta0 = np.random.normal(0, 0.01, size = (ncols, (ncols - 1)))
        self.theta1 = np.random.normal(0, 0.01, size = (ncols , (ncols - 1)))
        self.theta2 = np.random.normal(0, 0.01, size = (ncols, 1))
        
        self.costs = []
        cost = self.multiplecost(X, y)
        self.costs.append(cost)
        costprev = cost + self.convergence_thres + 1
        
        counter = 0
        
        for counter in np.arange(self.maxepochs):
            l1, l2, l3 = self.feedforward(X)
            
            # Backpropagation
            #l3_delta = (y - l3) * l3 * (1 - l3)
            #l2_delta =  l3_delta.T.dot(self.theta2.T) * l2 * (1 - l2)
            #l1_delta = l2_delta.T.dot(self.theta1.T) * l1 * (1 - l1)
            l3_delta = l3 - y
            l2_delta = l3_delta.T.dot(self.theta2.T) * l2 * (1 - l2)
            l1_delta = (l2_delta.dot(self.theta1)).T.dot((l1 * (1 - l1)))
            # Update parameters
            self.theta2 += l2.T.dot(l3_delta.T) / nobs * self.learning_rate
            self.theta1 += (l1 * l2_delta) / (nobs * self.learning_rate)
            self.theta0 += X.dot(l1_delta.T) / nobs * self.learning_rate
            
            counter += 1
            costprev = cost
            cost = self.multiplecost(X, y)
            self.costs.append(cost)
            if np.abs(costprev - cost) < self.convergence_thres and counter > 500:
                break
        

In [ ]:
import math
split = math.floor(X.shape[0] * .8)
X_train = X[:split]
y_train = y[:split]
X_test = X[split:]
y_test = y[split:]


learning_rate = 0.5
maxepochs = 10000
convergence_thres = 0.00001

_model = NNet4(learning_rate = learning_rate, maxepochs = maxepochs, convergence_thres = convergence_thres)

%time _model.learn(X_train, y_train)
probabilities_test = _model.predict(X_test)
probabilities_train = _model.predict(X_train)
auc_train = roc_auc_score(y_train, probabilities_train[0])
auc_test = roc_auc_score(y_test, probabilities_test[0])
print("Train score : {0} \n Test score : {1}".format(auc_train, auc_test))

In [ ]:
learning_rate = 0.5
maxepochs = 10000
convergence_thres = 0.00001

model = NNet3(learning_rate = learning_rate, maxepochs = maxepochs, convergence_thres = convergence_thres)

%time model.learn(X, y)

plt.plot(model.costs)
plt.title("Convergence of the cost function for a two layers NNets")
plt.ylabel("J($\Theta$)")
plt.xlabel("Iteration")
plt.legend();

### Decision Trees

In [7]:
_bank_data = bank_data[new_features_bis]
shuffled_index = np.random.permutation(_bank_data.index)
_bank_data = _bank_data.loc[shuffled_index]
split = math.floor(len(_bank_data.index) * .7)
train = _bank_data[:split]
test = _bank_data[split:]
_bank_data.tail()

,8_diffSumClosing stocks(kmt),6_diffSumClosing stocks(kmt),county_58,county_53,county_19,county_34,county_20,4_diffSumClosing stocks(kmt),2_diffSumClosing stocks(kmt),10_diffSumImports(kmt),...,7_diffSumProduction(kmt),12_diffSumExports(kmt),7_diffSumRefinery intake(kmt),1_diffSumImports(kmt),12_diffSumRefinery intake(kmt),1_diffSumRefinery intake(kmt),12_diffSumProduction(kmt),1_diffSumExports(kmt),1_diffSumProduction(kmt),actual_labels
6594,0.143562,0.094602,0.0,0.0,0.0,0.0,0.0,-0.065841,0.098136,1.727903,...,0.615302,0.890394,-2.394700,-0.975785,0.477530,-0.194815,0.587627,-1.261073,-0.556181,0
3250,0.747913,0.350922,0.0,0.0,0.0,0.0,0.0,-0.248736,1.160396,0.619888,...,-0.621748,-1.097692,-0.532956,1.631983,-1.678719,1.323856,-2.149698,0.194216,1.733688,0
5716,0.150653,0.634920,0.0,0.0,0.0,0.0,0.0,0.695167,0.261343,1.351669,...,-0.596103,-0.844547,-0.430863,0.681115,-0.645563,0.452519,-1.040750,0.904104,0.962915,1
5059,1.197743,0.101970,0.0,0.0,0.0,0.0,0.0,-0.219961,-0.341169,-0.275669,...,0.688586,1.397428,0.295187,-1.286743,0.107410,-0.514232,0.811423,-1.199302,-1.154385,1
73,0.302462,0.357674,0.0,0.0,0.0,0.0,0.0,0.141038,0.078800,1.214742,...,-0.793270,-0.875125,-0.320963,-0.080391,-0.730599,-0.757080,-0.544334,0.116534,0.679002,1


In [8]:
from sklearn.tree import DecisionTreeClassifier

tree_count = 30

bag_proportion = .6

predictions = []
train_predictions = []

for i in np.arange(tree_count):
    bag = _bank_data.sample(frac = bag_proportion, random_state = i, replace = True)
    clf = DecisionTreeClassifier(random_state = 1, splitter = 'random', max_features = 'auto', max_depth = 50,
                                min_samples_leaf = 2)
    clf.fit(bag[new_features], bag['actual_labels'])
    predictions.append(clf.predict_proba(bank_test[new_features]))
    train_predictions.append(clf.predict_proba(train[new_features]))
    

average_predictions = ((np.sum(predictions, axis = 0)) / tree_count)[:, 1]
train_average_predictions = ((np.sum(train_predictions,  axis = 0)) / tree_count)[:, 1]
auc_train = roc_auc_score(train['actual_labels'], train_average_predictions)
auc_train

0.89776834529530891

In [9]:
from sklearn.ensemble import RandomForestClassifier
clf_ens = RandomForestClassifier(n_estimators = 30, random_state = 1, min_samples_leaf = 1, max_depth = 30)
clf_ens.fit(_bank_data[new_features], _bank_data['actual_labels'])

predictions = clf_ens.predict(bank_test[new_features])
train_predictions = clf_ens.predict(train[new_features])


#auc_score = roc_auc_score(test['actual_labels'], predictions)
train_auc_score = roc_auc_score(train['actual_labels'], train_predictions)
print("train score {0} ".format(train_auc_score  ))

train score 0.8709356081809035 


In [10]:
from sklearn.tree import DecisionTreeClassifier

tree_count = 30

bag_proportion = .6

test_predictions = []
train_predictions = []
real_predictions = []

for i in np.arange(tree_count):
    bag = train.sample(frac = bag_proportion, random_state = i, replace = True)
    clf = DecisionTreeClassifier(random_state = 1, splitter = 'random', max_features = 'auto', max_depth = 50,
                                min_samples_leaf = 2)
    clf.fit(bag[new_features], bag['actual_labels'])
    test_predictions.append(clf.predict_proba(test[new_features]))
    train_predictions.append(clf.predict_proba(train[new_features]))
    #real_predictions.append(clf.predict_proba(bank_test[test_labels]))
    

average_predictions = ((np.sum(test_predictions, axis = 0)) / tree_count)[:, 1]
train_average_predictions = ((np.sum(train_predictions,  axis = 0)) / tree_count)[:, 1]
#average_predictions_test = ((np.sum(real_predictions,  axis = 0)) / tree_count)[:, 1]
auc_test = roc_auc_score(test['actual_labels'], average_predictions)
auc_train = roc_auc_score(train['actual_labels'], train_average_predictions)
print("train score : {0} \n test score : {1}".format(auc_train , auc_test))

train score : 0.8843166967355461 
 test score : 0.7727229961506058


In [35]:
output = pd.concat([bank_test['ID'], pd.DataFrame({'Target':probabilities[0]})], axis = 1)

In [36]:
output.to_csv('/Users/Kenneth-Aristide/anaconda3/bin/python_prog/ML_Training/data/NN_predictions.csv', index = False)